In [10]:
# %%writefile core.py
import numba as nb
import pandas as pd
import numpy as np
from loguru import logger
import numexpr as ne
import sympy
import scipy
from math import * 
logger.info(f"Pandas version: {pd.__version__}")
logger.info(f"Numpy version: {np.__version__}")
logger.info(f"Numba version: {ne.__version__}")
logger.info(f"NumExpr version: {nb.__version__}")
logger.info(f"SymPy version: {sympy.__version__}")
logger.info(f"SciPy version: {scipy.__version__}")

def softplus_np(n):
    x = np.random.uniform(50,150,size = n).astype(np.float32)
    safe = x < 100
    return np.log(1 + np.exp(x * safe)) * safe + x * (1-safe)

def softplus_vec_wrap(n):
    x = np.random.uniform(50,150,size = n).astype(np.float32)
    @nb.vectorize('float32(float32)',target = "parallel")
    def softplus_nb(x):
        safe = x < 100
        return log(1 + exp(x * safe)) * safe + x * (1-safe)
    result = softplus_nb(x)
    return result

softplus_jit = nb.jit(softplus_np)
softplus_njit = nb.njit(softplus_np)
softplus_fastmath_njit = nb.njit(fastmath=True,parallel=True)(softplus_np)


params = {"n":2000000}
softplus_np(2000000)
softplus_jit(2000000)
softplus_njit(2000000)
softplus_fastmath_njit(2000000)
softplus_vec_wrap(**params)

2021-02-13 18:10:18.255 | INFO     | __main__:<module>:9 - Pandas version: 1.1.3
2021-02-13 18:10:18.256 | INFO     | __main__:<module>:10 - Numpy version: 1.19.2
2021-02-13 18:10:18.257 | INFO     | __main__:<module>:11 - Numba version: 2.7.1
2021-02-13 18:10:18.258 | INFO     | __main__:<module>:12 - NumExpr version: 0.51.2
2021-02-13 18:10:18.259 | INFO     | __main__:<module>:13 - SymPy version: 1.6.2
2021-02-13 18:10:18.260 | INFO     | __main__:<module>:14 - SciPy version: 1.5.2


array([147.38025 , 140.12262 ,        inf, ..., 132.44917 , 113.141205,
       135.83322 ], dtype=float32)

In [11]:
params = {"n":2000000}
%time softplus_np(2000000)
%time softplus_jit(2000000)
%time softplus_njit(2000000)
%time softplus_fastmath_njit(2000000)
%time softplus_vec_wrap(**params)

CPU times: user 48.2 ms, sys: 12 ms, total: 60.3 ms
Wall time: 59.4 ms
CPU times: user 78.8 ms, sys: 3.92 ms, total: 82.7 ms
Wall time: 82.4 ms
CPU times: user 72.6 ms, sys: 23 µs, total: 72.6 ms
Wall time: 72.1 ms
CPU times: user 72.6 ms, sys: 265 µs, total: 72.9 ms
Wall time: 26.3 ms
CPU times: user 167 ms, sys: 0 ns, total: 167 ms
Wall time: 129 ms


array([146.32889, 112.83877,  72.02653, ..., 109.9768 , 100.81169,
       148.04533], dtype=float32)